In [1]:
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import torch.utils.data as Data
import torch

In [2]:
batch_size = 200
learning_rate = 0.01

train_data = datasets.MNIST('../../../../Other/datasets/PyTorch',
                            train=True,
                            transform=transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))
                            ]))

test_data = datasets.MNIST('../../../../Other/datasets/PyTorch',
                           train=False,
                           transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,))
                           ]))

# 手写数字训练数据集
train_loader = Data.DataLoader(dataset=train_data,
                               batch_size=batch_size,
                               shuffle=True)

# 手写数字测试数据集
test_loader = Data.DataLoader(dataset=test_data,
                              batch_size=batch_size,
                              shuffle=True)

In [3]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.featrues = nn.Sequential(  # 内部实现了forward函数;各模块顺序执行
            nn.Conv2d(1, 6, 3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(6, 16, 3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(400, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10),  # 10分类问题
        )

    def forward(self, x):
        x = self.featrues(x)
        x = x.reshape(x.size()[0], -1)
        x = self.classifier(x)
        return x

In [4]:
criteon = nn.CrossEntropyLoss()  # 定义损失函数为交叉熵
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = LeNet()
lenet = net.to(device)  # 模型设备转移必须在优化器定义前执行

# RMSprop优化器
optimizer = optim.RMSprop(net.parameters(),
                          lr=learning_rate,  # 学习率
                          momentum=0.4,  # 动量参数\alpha;默认momentum=0
                          alpha=0.9,  # 衰减速率\rho
                          eps=1e-10)  # 分母中的小常数\delta

In [5]:
transforms_data = []
for i, j in train_data:
    transforms_data.append(i.tolist())
transforms_data_ten = torch.tensor(transforms_data)
transforms_data_ten

tensor([[[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          ...,
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242]]],


        [[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          ...,
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242]]],


        [[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.424

In [6]:
transforms_data_loader = Data.DataLoader(transforms_data_ten, batch_size=200, shuffle=True)
y_label = train_loader.dataset.targets

In [7]:
from torch_model.train_evaluate import Train_Evaluate

t_and_v = Train_Evaluate(model=lenet, optimizer=optimizer, criterion=criteon, epochs=5)

In [8]:
metric_reslut = t_and_v.train_eval(train_loader=train_loader, valid_sets=[(transforms_data_loader, y_label)])
metric_reslut

C:\Users\duanm\anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Train Epoch: 0  [0    /60000 (0  %)]	Loss: 2.306799
Train Epoch: 0  [2000 /60000 (3  %)]	Loss: 2.219586
Train Epoch: 0  [4000 /60000 (7  %)]	Loss: 1.588877
Train Epoch: 0  [6000 /60000 (10 %)]	Loss: 1.532783
Train Epoch: 0  [8000 /60000 (13 %)]	Loss: 0.637401
Train Epoch: 0  [10000/60000 (17 %)]	Loss: 0.555727
Train Epoch: 0  [12000/60000 (20 %)]	Loss: 0.254109
Train Epoch: 0  [14000/60000 (23 %)]	Loss: 0.319782
Train Epoch: 0  [16000/60000 (27 %)]	Loss: 0.312815
Train Epoch: 0  [18000/60000 (30 %)]	Loss: 0.216359
Train Epoch: 0  [20000/60000 (33 %)]	Loss: 0.307836
Train Epoch: 0  [22000/60000 (37 %)]	Loss: 0.221181
Train Epoch: 0  [24000/60000 (40 %)]	Loss: 0.111998
Train Epoch: 0  [26000/60000 (43 %)]	Loss: 0.202543
Train Epoch: 0  [28000/60000 (47 %)]	Loss: 0.229926
Train Epoch: 0  [30000/60000 (50 %)]	Loss: 0.113203
Train Epoch: 0  [32000/60000 (53 %)]	Loss: 0.224661
Train Epoch: 0  [34000/60000 (57 %)]	Loss: 0.241728
Train Epoch: 0  [36000/60000 (60 %)]	Loss: 0.223739
Train Epoch:

[[12.682374000549316],
 [19.897192001342773],
 [21.733537673950195],
 [21.752864837646484],
 [32.318443298339844]]